In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import random
import time
from collections import deque
from datetime import datetime, timedelta

import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots

from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score

class AdvancedAdPricingDashboard:
    def __init__(self, max_buffer_size=5000):
        # Enhanced database and data management
        self.conn = sqlite3.connect('advanced_ad_pricing.db', check_same_thread=False)
        self.cursor = self.conn.cursor()
        self._create_database_schema()

        # Advanced data management
        self.data_buffer = deque(maxlen=max_buffer_size)
        self.ml_models = {
            'global': RandomForestRegressor(n_estimators=150, max_depth=15, random_state=42),
            'location_specific': {}
        }

        # Enhanced preprocessing
        self.scaler = StandardScaler()
        self.label_encoders = {
            'location': LabelEncoder(),
            'season': LabelEncoder(),
            'campaign_type': LabelEncoder()
        }

        # Performance tracking
        self.performance_metrics = {
            'global_model': {'mae': [], 'r2': []},
            'location_models': {}
        }

        # Initialize data and models
        self._initialize_system()

    def _create_database_schema(self):
        """Create an enhanced database schema with more detailed tracking."""
        self.cursor.execute('''
        CREATE TABLE IF NOT EXISTS ad_pricing (
            id INTEGER PRIMARY KEY,
            timestamp TEXT,
            location TEXT,
            season TEXT,
            campaign_type TEXT,
            income_strength REAL,
            user_engagement_score REAL,
            demographic_density INTEGER,
            time_of_day INTEGER,
            day_of_week INTEGER,
            historical_cpc REAL,
            predicted_cpc REAL
        )
        ''')
        self.conn.commit()

    def _generate_synthetic_data(self, num_records=50):
        """Generate more sophisticated synthetic ad pricing data."""
        campaign_types = ['Performance', 'Brand Awareness', 'Conversion', 'Retargeting']
        locations = ['Mumbai', 'Delhi', 'Bengaluru', 'Chennai', 'Kolkata',
                     'Hyderabad', 'Pune', 'Ahmedabad', 'Surat', 'Jaipur']

        data = []
        for _ in range(num_records):
            location = random.choice(locations)
            base_cpc = {
                'Mumbai': random.uniform(50, 250),
                'Delhi': random.uniform(40, 220),
                'Bengaluru': random.uniform(60, 280),
                'Chennai': random.uniform(45, 200),
                'Kolkata': random.uniform(35, 180),
                'Hyderabad': random.uniform(40, 210),
                'Pune': random.uniform(50, 240),
                'Ahmedabad': random.uniform(45, 190),
                'Surat': random.uniform(30, 150),
                'Jaipur': random.uniform(35, 170)
            }[location]

            record = (
                datetime.now().isoformat(),
                location,
                random.choice(['Spring', 'Summer', 'Fall', 'Winter']),
                random.choice(campaign_types),
                random.uniform(0.1, 1.0),  # income_strength
                random.uniform(0.1, 1.0),  # user_engagement_score
                random.randint(1000, 10000),  # demographic_density
                random.randint(0, 23),  # time_of_day
                random.randint(0, 6),  # day_of_week
                base_cpc,  # historical_cpc
                base_cpc * (1 + random.uniform(-0.2, 0.2))  # predicted_cpc with variation
            )
            data.append(record)

        return data

    def _initialize_system(self):
        """Enhanced system initialization with more robust data processing."""
        # Pre-populate label encoders
        locations = ['Mumbai', 'Delhi', 'Bengaluru', 'Chennai', 'Kolkata',
                     'Hyderabad', 'Pune', 'Ahmedabad', 'Surat', 'Jaipur']
        campaign_types = ['Performance', 'Brand Awareness', 'Conversion', 'Retargeting']

        self.label_encoders['location'].fit(locations)
        self.label_encoders['season'].fit(['Spring', 'Summer', 'Fall', 'Winter'])
        self.label_encoders['campaign_type'].fit(campaign_types)

        # Generate and insert initial data
        initial_data = self._generate_synthetic_data(200)
        self.cursor.executemany('''
        INSERT INTO ad_pricing
        (timestamp, location, season, campaign_type, income_strength, user_engagement_score,
        demographic_density, time_of_day, day_of_week, historical_cpc, predicted_cpc)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', initial_data)
        self.conn.commit()

        # Load and process data
        self._load_data()
        self._train_models()

    def _load_data(self):
        """Load and process data from the database."""
        query = "SELECT * FROM ad_pricing"
        df = pd.read_sql_query(query, self.conn)

        # Process and clean data
        df = self._preprocess_data(df)

        # Update data buffer
        self.data_buffer.extend(df.to_dict('records'))

    def _preprocess_data(self, df):
        """Advanced data preprocessing with feature engineering."""
        # Handle categorical encoding
        for col, encoder in self.label_encoders.items():
            df[col + '_encoded'] = encoder.transform(df[col])

        # Feature engineering
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        df['hour_of_day'] = df['timestamp'].dt.hour
        df['month'] = df['timestamp'].dt.month

        return df

    def _train_models(self):
        """Train global and location-specific models."""
        df = pd.DataFrame(list(self.data_buffer))

        # Prepare features and target
        features = [
            'income_strength', 'user_engagement_score', 'demographic_density',
            'time_of_day', 'day_of_week', 'location_encoded',
            'season_encoded', 'campaign_type_encoded'
        ]

        X = df[features]
        y = df['historical_cpc']

        # Global model
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        self.ml_models['global'].fit(X_train, y_train)

        # Performance metrics for global model
        y_pred = self.ml_models['global'].predict(X_test)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        self.performance_metrics['global_model']['mae'].append(mae)
        self.performance_metrics['global_model']['r2'].append(r2)

        # Location-specific models
        for location in df['location'].unique():
            location_data = df[df['location'] == location]
            if len(location_data) > 50:  # Minimum data requirement
                X_loc = location_data[features]
                y_loc = location_data['historical_cpc']

                loc_model = RandomForestRegressor(n_estimators=100, max_depth=10)
                X_train_loc, X_test_loc, y_train_loc, y_test_loc = train_test_split(X_loc, y_loc, test_size=0.2)

                loc_model.fit(X_train_loc, y_train_loc)
                self.ml_models['location_specific'][location] = loc_model

    def create_pricing_dashboard(self):
        """Create an advanced, interactive dashboard."""
        df = pd.DataFrame(list(self.data_buffer))

        # Create subplot layout
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=[
                'Average CPC by Location',
                'CPC Distribution',
                'Location Performance',
                'CPC Trend Over Time'
            ],
            specs=[
                [{'type':'bar'}, {'type':'box'}],
                [{'type':'pie'}, {'type':'scatter'}]
            ]
        )

        # Bar chart: Average CPC by Location
        location_cpc = df.groupby('location')['historical_cpc'].mean().sort_values(ascending=False)
        fig.add_trace(
            go.Bar(
                x=location_cpc.index,
                y=location_cpc.values,
                name='Average CPC',
                marker_color='royalblue'
            ),
            row=1, col=1
        )

        # Box plot: CPC Distribution
        fig.add_trace(
            go.Box(
                y=df['historical_cpc'],
                name='CPC Distribution',
                boxpoints='outliers'
            ),
            row=1, col=2
        )

        # Pie chart: Campaign Type Distribution
        campaign_dist = df['campaign_type'].value_counts()
        fig.add_trace(
            go.Pie(
                labels=campaign_dist.index,
                values=campaign_dist.values,
                hole=0.4
            ),
            row=2, col=1
        )

        # Scatter: CPC Trend
        fig.add_trace(
            go.Scatter(
                x=pd.to_datetime(df['timestamp']),
                y=df['historical_cpc'],
                mode='lines+markers',
                name='CPC Trend'
            ),
            row=2, col=2
        )

        # Update layout
        fig.update_layout(
            height=900,
            title_text='Advanced Ad Pricing Dashboard',
            showlegend=True,
            template='plotly_white'
        )

        return fig

    def run_simulation(self, interval_seconds=20):
        """Run continuous simulation and model refinement."""
        while True:
            # Generate new data
            new_data = self._generate_synthetic_data(50)

            # Insert into database
            self.cursor.executemany('''
            INSERT INTO ad_pricing
            (timestamp, location, season, campaign_type, income_strength, user_engagement_score,
            demographic_density, time_of_day, day_of_week, historical_cpc, predicted_cpc)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', new_data)
            self.conn.commit()

            # Reload and retrain
            self._load_data()
            self._train_models()

            # Create dashboard
            dashboard = self.create_pricing_dashboard()
            dashboard.show()

            # Wait before next iteration
            time.sleep(interval_seconds)

# Run the simulation
dashboard = AdvancedAdPricingDashboard()
dashboard.run_simulation()